# Department of Transportation - Oil Spill Mapping

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import openpyxl
import folium
import json
import gmaps.datasets
import numpy as np
# from shapely.geometry import Point, Polygon
# import geopandas as gpd
# import descartes
import matplotlib.pyplot as plt

%matplotlib inline

# Import API key
from config import g_map

# Configure gmaps
gmaps.configure(api_key=g_map)

## Bring in 2010 - Present Data

In [4]:
# 1. Read the Data_Columns_and_spills into a DataFrame. All sheets
# Unedited excel files
xlfile_2010 = pd.ExcelFile("../Resources/Data_columns_and_spills.xlsx", engine='openpyxl')
spill2010_df = xlfile_2010.parse('2010-Present') #'2010-Present' is the sheet name in the excel file
#spill2010_df is all data all columns
spill2010_df

REPORT_RECEIVED_DATE  IYEAR  REPORT_NUMBER  SUPPLEMENTAL_NUMBER  \
0              2018-12-10   2018       20180382                32252   
1              2020-11-06   2020       20200300                34501   
2              2015-02-24   2015       20150071                21327   
3              2020-09-13   2020       20200253                35187   
4              2015-02-06   2015       20150040                20630   
...                   ...    ...            ...                  ...   
4635           2018-01-08   2017       20180006                23089   
4636           2018-05-30   2018       20180156                30565   
4637           2019-01-03   2018       20190001                31424   
4638           2019-01-24   2018       20190021                33804   
4639           2021-03-11   2021       20210060                34982   

             REPORT_TYPE  OPERATOR_ID                               NAME  \
0     SUPPLEMENTAL FINAL        31130                      DCP MIDSTREAM   
1         ORIGINAL FINAL        32545  DENBURY GULF COAST PIPELINES, LLC   
2     SUPPLEMENTAL FINAL        31618  ENTERPRISE PRODUCTS OPERATING LLC   
3           SUPPLEMENTAL         2552               COLONIAL PIPELINE CO   
4     SUPPLEMENTAL FINAL        31618  ENTERPRISE PRODUCTS OPERATING LLC   
...                  ...          ...                                ...   
4635  SUPPLEMENTAL FINAL         3445         DIXIE PIPELINE COMPANY LLC   
4636  SUPPLEMENTAL FINAL        30829      ENTERPRISE CRUDE PIPELINE LLC   
4637      ORIGINAL FINAL        39145  ENBRIDGE STORAGE (CUSHING) L.L.C.   
4638  SUPPLEMENTAL FINAL          300              PLAINS PIPELINE, L.P.   
4639      ORIGINAL FINAL          300              PLAINS PIPELINE, L.P.   

               OPERATOR_STREET_ADDRESS OPERATOR_CITY_NAME  \
0           370 17TH STREET SUITE 2500             DENVER   
1        5851 LEGACY CIRCLE SUITE 1200              PLANO   
2               1100 LOUISIANA STREET             HOUSTON   
3     1185 SANCTUARY PARKWAY SUITE 100         ALPHARETTA   
4               1100 LOUISIANA STREET             HOUSTON   
...                                ...                ...   
4635               1100 LOUISIANA ST.             HOUSTON   
4636            1100 LOUISIANA STREET             HOUSTON   
4637            5400 WESTHEIMER COURT             HOUSTON   
4638     333 CLAY STREET, SUITE 1900              HOUSTON   
4639     333 CLAY STREET, SUITE 1900              HOUSTON   

     OPERATOR_STATE_ABBREVIATION  ...                       PREPARER_TITLE  \
0                             CO  ...   SR. PIPELINE COMPLIANCE SPECIALIST   
1                             TX  ...     REGULATORY COMPLIANCE SPECIALIST   
2                             TX  ...            SR. COMPLIANCE SPECIALIST   
3                             GA  ...        MANAGER - PIPELINE COMPLIANCE   
4                             TX  ...   SR. PIPELINE COMPLIANCE SPECIALIST   
...                          ...  ...                                  ...   
4635                          TX  ...         PIPELINE COMPLIANCE ENGINEER   
4636                          TX  ...  PIPELINE COMPLIANCE SENIOR ENGINEER   
4637                          TX  ...                SR COMPLIANCE ANALYST   
4638                          TX  ...            ERC COMPLIANCE SPECIALIST   
4639                          TX  ...            HSE COMPLIANCE SPECIALIST   

                PREPARER_EMAIL  PREPARER_TELEPHONE  PREPARER_FAX  \
0     kdneden@dcpmidstream.com        979-242-6976           NaN   
1     chad.docekal@denbury.com        972-673-2734           NaN   
2           mjpavlak@eprod.com        713-381-5897           NaN   
3          mpiazza@colpipe.com        678-763-5911           NaN   
4          wnburrell@eprod.com        713-381-3536           NaN   
...                        ...                 ...           ...   
4635      lswilliams@eprod.com        713-381-6748           NaN   
4636      lsw

In [5]:
# Pulling useful columns for mapping purposes
my_columns = [
    'REPORT_NUMBER',
    'OPERATOR_ID',
    'LOCATION_LATITUDE',
    'LOCATION_LONGITUDE',
    'COMMODITY_RELEASED_TYPE',
    'UNINTENTIONAL_RELEASE_BBLS',
    'ON_OFF_SHORE',
    'ONSHORE_CITY_NAME',
    'ONSHORE_COUNTY_NAME',
    'ONSHORE_STATE_ABBREVIATION',
    'SYSTEM_PART_INVOLVED',
    'INSTALLATION_YEAR',
    'CAUSE',
    ]

In [6]:
# mapping columns called info2010_df
info2010_df = spill2010_df[my_columns]
# Creating another dataframe of first 100 rows called 'test' t2010_df
# Used to get map functioning and limiting API requests
t2010_df = info2010_df[:100]
t2010_df.to_json('test_df.json') # exports DataFrame to json for javascript and mapping functionality
t2010_df

REPORT_NUMBER  OPERATOR_ID  LOCATION_LATITUDE  LOCATION_LONGITUDE  \
0        20180382        31130          30.351556         -100.581792   
1        20200300        32545          32.628233          -90.569264   
2        20150071        31618          40.366817          -80.584594   
3        20200253         2552          35.414106          -80.806185   
4        20150040        31618          28.885881          -96.433027   
..            ...          ...                ...                 ...   
95       20100150        32296          30.259890          -93.209910   
96       20120009        22610          40.312770          -95.721250   
97       20100130          395          41.844019          -85.741131   
98       20170243        39504          30.085836          -97.386029   
99       20140216        31555          31.115800         -102.293100   

                              COMMODITY_RELEASED_TYPE  \
0   HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...   
1                                CO2 (CARBON DIOXIDE)   
2   HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...   
3   REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...   
4   HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...   
..                                                ...   
95  HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...   
96  REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...   
97  REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...   
98                                          CRUDE OIL   
99                               CO2 (CARBON DIOXIDE)   

    UNINTENTIONAL_RELEASE_BBLS ON_OFF_SHORE          ONSHORE_CITY_NAME  \
0                      48400.0      ONSHORE  Not Within a Municipality   
1                      41177.0      ONSHORE  Not Within a Municipality   
2                      30565.0      ONSHORE                   COLLIERS   
3                      28571.0      ONSHORE               HUNTERSVILLE   
4                      27123.0      ONSHORE  Not Within a Municipality   
..                         ...          ...                        ...   
95                      2237.0      ONSHORE               LAKE CHARLES   
96                      2184.0      ONSHORE                NEMAHA CITY   
97                      2121.0      ONSHORE               CONSTANTINE    
98                      2084.0      ONSHORE  Not Within a Municipality   
99                      2066.0      ONSHORE                    MCCAMEY   

   ONSHORE_COUNTY_NAME ONSHORE_STATE_ABBREVIATION  \
0               SUTTON                         TX   
1         YAZOO COUNTY                         MS   
2               BROOKE                         WV   
3          MECKLENBURG                         NC   
4              JACKSON                         TX   
..                 ...                        ...   
95           CALCASIEU                         LA   
96               NEMAH                         NE   
97          ST. JOSEPH                         MI   
98             BASTROP                         TX   
99               UPTON                         TX   

                               SYSTEM_PART_INVOLVED  INSTALLATION_YEAR  \
0           ONSHORE PIPELINE, INCLUDING VALVE SITES             2013.0   
1           ONSHORE PIPELINE, INCLUDING VALVE SITES             2009.0   
2           ONSHORE PIPELINE, INCLUDING VALVE SITES             2013.0   
3           ONSHORE PIPELINE, INCLUDING VALVE SITES             1978.0   
4           ONSHORE PIPELINE, INCLUDING VALVE SITES             1966.0   
..                                              ...                ...   
95          ONSHORE PIPELINE, INCLUDING VALVE SITES                NaN   
96          ONSHORE PIPELINE, INCLUDING VALVE SITES             1950.0   
97          ONSHORE PIPELINE, INCLUDING VALVE SITES             1953.0   
98          ONSHORE PIPELINE, INCLUDING VALVE SITES                NaN   
99  ONSHORE PUMP/METER STATION EQUIPMENT AND PIPING             2010.0   

                           

## Mapping 2010-Present 'info2010_df' spill locations ~4000 locations

In [10]:
info_box_template = """
<dl>
<dt>Operator ID: </dt><dd>{OPERATOR_ID}</dd>
<dt>City</dt><dd>{ONSHORE_CITY_NAME}</dd>
<dt>State</dt><dd>{ONSHORE_STATE_ABBREVIATION}</dd>
<dt>Spill Commodity</dt><dd>{COMMODITY_RELEASED_TYPE}</dd>
<dt>Spill Volume (bbls)</dt><dd>{UNINTENTIONAL_RELEASE_BBLS}</dd>
</dl>

"""


spill_info = [info_box_template.format(**row) for index, row in info2010_df.iterrows()]

In [11]:
# Create map from lat and long
locations = info2010_df[["LOCATION_LATITUDE","LOCATION_LONGITUDE"]]
spill = info2010_df["UNINTENTIONAL_RELEASE_BBLS"]
fig = gmaps.figure(center=(36, -97), zoom_level=4)
heat_layer = gmaps.heatmap_layer(locations, weights=spill,dissipating=False,
            max_intensity=30000, point_radius=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=spill_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

# NEED TO GET WITH TA'S TO SEE IF REFACTORING THIS CODE FOR BATCH API CALLS IS POSSIBLE WITH GOOGLE API'S

## Using Batch API Calls to Improve Performance

### Batch API calls are one of the easiest ways to improve the performance of your code

### This is because HTTP requests are typically one of the slowest components of a script
    
### Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to

In [ ]:
# Function sourced from
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):\n",
    """Yield successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
    spill_groups = list(chunks(info2010_df['REPORT_NUMBER'], 500))
    spill_strings = []
for i in range(0, len(spill_groups)):
    spill_strings.append(''.join(str(spill_groups)[1:-1]))
    print(spill_strings[i])
    
for spill_string in spill_strings:
    print(spill_string)

## Bring in 2002-2010 Data

In [12]:
# 1. Read the Data_Columns_and_spills into a DataFrame. All sheets
# Unedited excel files
xlfile_2002 = pd.ExcelFile("../Resources/Data_columns_and_spills.xlsx", engine='openpyxl')
spill2002_df = xlfile_2002.parse('2002-2010') #'2002-2010' is the sheet name in the excel file
#spill2002_df is all data all columns
spill2002_df

IYEAR  OPERATOR_ID     RPTID  OWNER_OPERATOR_ID  \
0      2002        19237  20020037                NaN   
1      2002        25146  20020038                NaN   
2      2002        12628  20020039                NaN   
3      2002         2731  20020040                NaN   
4      2002         4472  20020041                NaN   
...     ...          ...       ...                ...   
3025   2009        10012  20090382                NaN   
3026   2007        32147  20090383                NaN   
3027   2006        32147  20090384                NaN   
3028   2009        32147  20090385                NaN   
3029   2009        22442  20090386                NaN   

                                            NAME  \
0                   TE PRODUCTS PIPELINE CO., LP   
1                       EQUISTAR CHEMICALS, L.P.   
2                              MOBIL PIPELINE CO   
3                            CHEVRON PIPELINE CO   
4            KINDER MORGAN ENERGY PARTNERS, L.P.   
...                                          ...   
3025  NUSTAR PIPELINE OPERATING PARTNERSHIP L.P.   
3026                      MARATHON PIPE LINE LLC   
3027                      MARATHON PIPE LINE LLC   
3028                      MARATHON PIPE LINE LLC   
3029                 WEST TEXAS GULF PIPELINE CO   

                                  OPSTREET      OPCITY   OPCOUNTY OPSTATE  \
0                       2929 ALLEN PARKWAY     HOUSTON     HARRIS      TX   
1                              PO BOX 1847       ALVIN   BRAZORIA      TX   
2                          800 BELL STREET     HOUSTON     HARRIS      TX   
3                          2811 HAYES ROAD     HOUSTON     HARRIS      TX   
4                500 DALLAS ST, SUITE 1000     HOUSTON     HARRIS      TX   
...                                    ...         ...        ...     ...   
3025     7340 W. 21ST STREET N., SUITE 200     WICHITA   SEDGWICK      KS   
3026                 539 SOUTH MAIN STREET     FINDLAY    HANCOCK      OH   
3027                 539 SOUTH MAIN STREET     FINDLAY    HANCOCK      OH   
3028                 539 SOUTH MAIN STREET     FINDLAY    HANCOCK      OH   
3029  ONE FLUOR DANIEL DR. BLDG. A LEVEL 3  SUGAR LAND  FORT BEND      TX   

      OPZIP  ...  IO_DRUG IO_ALCO  \
0     77019  ...      NaN     NaN   
1     77512  ...      NaN     NaN   
2     77002  ...      0.0     0.0   
3     77082  ...      NaN     NaN   
4     77002  ...      NaN     NaN   
...     ...  ...      ...     ...   
3025  67205  ...      NaN     NaN   
3026  45840  ...      0.0     0.0   
3027  45840  ...      0.0     0.0   
3028  45840  ...      NaN     NaN   
3029  77478  ...      NaN     NaN   

                                                   MISC UNKNOWN UNKNOWN_TXT  \
0                                                   NaN     NaN         NaN   
1                                                   NaN     NaN         NaN   
2                                                   NaN     NaN         NaN   
3                                                   NaN     NaN         NaN   
4                                                   NaN     NaN         NaN   
...                                                 ...     ...         ...   
3025                                                NaN     NaN         NaN   
3026                                                NaN     NaN         NaN   
3027                                                NaN     NaN         NaN   
3028  RESIDUAL PRODUCT IN THE LINE IGNITED FOLLOWING...     NaN         NaN   
3029  IGNITED VAPORS WHILE BEVELING PIPE WITH AN ACE...     NaN         NaN   

                                                  PNAME        PTEL  \
0                                           JLYNN STOUT  7137593614   
1                                          GREG LEBLANC  7138446942   
2                                     STEVEN J ROBENOLT  7136562651   
3                                            R A THORPE  2815963571   
4                     

In [13]:
my_columns2002 = [
    'RPTID',
    'OPERATOR_ID',
    'LATITUDE',
    'LONGITUDE',
    'CLASS_TXT',
    'LOSS',
    'SPUNIT_TXT',
    'OFFSHORE',
    'ACCITY',
    'ACCOUNTY',
    'ACSTATE',
    'SYSPRT_TXT',
    'PRTYR',
    'GEN_CAUSE_TXT'
   ]

### I need to convert lat and long and gallons to bbls before mapping...gimmie a day or two